In [1]:
# --- Setup & Imports ---

import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from typing import Dict, Any, List, Tuple, Optional
import json

from lightgbm import LGBMRegressor  # Direct LightGBM import
import lightgbm as lgb

# --- 0. GPU Setup (LightGBM) -------------------------------------------------

USE_GPU_PREFERRED = True          # Use GPU if available?
GPU_PLATFORM_ID = 0               # OpenCL platform id
GPU_DEVICE_ID = 0                 # OpenCL device id


def _try_get_opencl_device_name(platform_id: int, device_id: int) -> Optional[str]:
    """Best-effort OpenCL device name lookup via pyopencl (optional)."""
    try:
        import pyopencl as cl  # optional
        plats = cl.get_platforms()
        if platform_id < 0 or platform_id >= len(plats):
            return None
        devs = plats[platform_id].get_devices()
        if device_id < 0 or device_id >= len(devs):
            return None
        return f"{plats[platform_id].name} | {devs[device_id].name}"
    except Exception:
        return None


def _probe_lightgbm_gpu(platform_id: int, device_id: int) -> Tuple[bool, str]:
    """Check whether LightGBM GPU training works in this environment."""
    X_probe = np.random.RandomState(0).randn(200, 10)
    y_probe = X_probe[:, 0] * 0.7 + np.random.RandomState(1).randn(200) * 0.1

    try:
        m = LGBMRegressor(
            n_estimators=20,
            learning_rate=0.1,
            max_depth=-1,
            num_leaves=31,
            device_type="gpu",
            gpu_platform_id=platform_id,
            gpu_device_id=device_id,
            n_jobs=1,
            random_state=42,
            verbose=-1,
        )
        m.fit(X_probe, y_probe)
        return True, "GPU probe training succeeded."
    except Exception as e:
        return False, f"GPU probe failed: {repr(e)}"


def configure_lgbm_device(prefer_gpu: bool, platform_id: int, device_id: int) -> Dict[str, Any]:
    """Decide once whether to use GPU or fall back to CPU; print status (and device name if available)."""
    print("\n" + "=" * 60)
    print("--- LightGBM Device Selection ---")
    print("=" * 60)
    print("LightGBM version:", getattr(lgb, "__version__", "unknown"))
    print("Prefer GPU:", bool(prefer_gpu))

    if not prefer_gpu:
        print("=> GPU is explicitly disabled. Using CPU.")
        return {"device_type": "cpu"}

    ok, msg = _probe_lightgbm_gpu(platform_id, device_id)
    if ok:
        dev_name = _try_get_opencl_device_name(platform_id, device_id)
        if dev_name:
            print("=> Using GPU:", dev_name)
        else:
            print(
                f"=> Using GPU (platform_id={platform_id}, device_id={device_id}); "
                "device name not available (pyopencl missing, etc.)."
            )
        print("Note:", msg)
        return {
            "device_type": "gpu",
            "gpu_platform_id": platform_id,
            "gpu_device_id": device_id,
        }

    print("=> GPU was requested, but is not available/usable.")
    print("   Reason:", msg)
    print("=> Fallback: using CPU.")
    return {"device_type": "cpu"}


LGBM_DEVICE_PARAMS = configure_lgbm_device(
    prefer_gpu=USE_GPU_PREFERRED,
    platform_id=GPU_PLATFORM_ID,
    device_id=GPU_DEVICE_ID,
)

# --- 1. Path setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()


NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# --- Import required functions and models ---
from src.config import GlobalConfig, DEFAULT_CORR_SPEC, OUTPUTS
from src.io_timesplits import load_target, load_ifo_features
from src.features import (
    build_engineered_matrix,
    redundancy_reduce_greedy,
    pw_corr,  # correlation-based lag selection
)
from src.models.ET import ForecastModel as ETModel
from sklearn.linear_model import ElasticNet  # local EN class only

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- Output directory ---
BASE_OUTPUT_DIR = OUTPUTS / "feature_importance"
OUTPUT_DIR = BASE_OUTPUT_DIR / "outputs_no_missing"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print("Output Directory =", OUTPUT_DIR)


# --- Local ENModel (ElasticNet wrapper) ---
class ENModel:
    """ElasticNet without CV; fixed alpha/l1_ratio; uses |coef| as importance."""
    def __init__(self, params: Optional[Dict[str, Any]] = None):
        self.params = dict(params or {})
        self._reg = None
        self._feature_names: Optional[List[str]] = None
        self._l1_ratio = float(self.params.get("alpha", 0.5))   # mixing parameter
        self._lambda   = float(self.params.get("lambda", 0.1))  # regularization strength

    @staticmethod
    def _clean(X):
        X = np.asarray(X, dtype=float)
        return np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0, copy=False)

    def fit(self, X, y, sample_weight=None):
        if isinstance(X, pd.DataFrame):
            self._feature_names = X.columns.tolist()
            X_np = X.values
        else:
            X_np = np.asarray(X)
            self._feature_names = [f"feature_{i}" for i in range(X_np.shape[1])]
        X_np = self._clean(X_np)
        y_np = np.asarray(y, dtype=float).ravel()
        self._reg = ElasticNet(
            alpha=self._lambda, l1_ratio=self._l1_ratio,
            random_state=42, fit_intercept=True
        )
        self._reg.fit(X_np, y_np, sample_weight=sample_weight)
        return self


# --- 2. Configuration ---
REDUNDANCY_THRESHOLD = 0.90        # redundancy filter for Mean selection
TOP_N_PER_MODEL = 20               # fixed Top-N per model for Output 1
MAX_LAG = 6                        # lags 0..6 (0 = current month, 1..6 past)

# --- Hyperparameters ---
HP_EN = {
    # as in tuning grid:
    "alpha": 0.0,        # mixing -> l1_ratio
    "lambda": 10,        # penalty -> sklearn alpha
    "seed": 42,
}
HP_ET = {
    "n_estimators": 1000,
    "max_depth": None,          # or None
    "min_samples_leaf": 10,
    "max_features": 0.25,       # or "sqrt"
    "seed": 42,
}
HP_LGBM = {
    # as in tuning grid:
    "n_estimators": 5000,
    "learning_rate": 0.02,
    "num_leaves": 31,
    "max_depth": 4,             # or -1
    "min_child_samples": 50,
    "feature_fraction": 0.8,
    "reg_lambda": 10.0,
    "min_child_weight": 0.001,
    "importance_type": "gain",
    "n_jobs": 1,
    "seed": 42,
}

# --- 3. Load data ---
y = load_target()
X_ifo = load_ifo_features()
idx = y.index.intersection(X_ifo.index)
y, X_ifo = y.loc[idx], X_ifo.loc[idx]
T = len(y)
print("Data loaded. Shapes:", X_ifo.shape, y.shape)


# --- 4a. Helper: best lag >0 per feature (plus lag 0) ---

def build_best_lag_map_global(
    X: pd.DataFrame,
    y: pd.Series,
    max_lag: int,
    corr_spec: Any,
) -> Dict[str, List[int]]:
    """
    For each feature:
    - compute correlation of lag ℓ (ℓ=1..max_lag) with y_{t+1}
    - select ℓ*>0 with max |corr|
    - return lag_map[col] = [0, ℓ*] (sorted)

    Lag 0 is always included; exactly one best lag >0 is added.
    """
    y_next = y.shift(-1)  # target y_{t+1}
    lag_map: Dict[str, List[int]] = {}

    for col in X.columns:
        s = X[col]
        best_lag = None
        best_score = -1.0

        for lag in range(1, max_lag + 1):
            x_lag = s.shift(lag)
            mask = (~x_lag.isna()) & (~y_next.isna())
            if mask.sum() < 3:
                continue

            score = abs(pw_corr(x_lag[mask].to_numpy(), y_next[mask].to_numpy(), corr_spec))
            if score > best_score:
                best_score = score
                best_lag = lag

        # Fallback if no valid correlation was computed
        if best_lag is None:
            best_lag = 1 if max_lag >= 1 else 0

        lags_for_col = sorted({0, int(best_lag)})
        lag_map[col] = lags_for_col

    return lag_map


# --- 4b. Feature pipeline (lag 0 + best lag >0, target y_{t+1}) ---

def run_feature_pipeline_for_importance(
    X: pd.DataFrame,
    y: pd.Series,
    t_origin: int,
    lag_map: Dict[str, List[int]],
) -> Tuple[pd.DataFrame, pd.Series, Dict[str, List[int]]]:
    """
    Build X_tr, y_tr for a given origin t_origin.

    - uses a global lag_map (lag 0 + best lag >0 per feature)
    - builds the lagged matrix
    - trims by required history and I_t
    - target is y_{t+1}
    """
    I_t = t_origin + 1  # training window ends at t_origin

    if lag_map is None or len(lag_map) == 0:
        return pd.DataFrame(), pd.Series(dtype=float), {}

    X_eng = build_engineered_matrix(X, lag_map)
    if X_eng.empty:
        return pd.DataFrame(), pd.Series(dtype=float), lag_map

    # max required lag from column names ("feature__lagℓ")
    def _lag_of(col):
        try:
            return int(str(col).split('__lag')[-1])
        except Exception:
            return 0

    head_needed = max([_lag_of(c) for c in X_eng.columns] + [0])

    # base time indices: 1 .. I_t-1
    taus = np.arange(1, int(I_t) - 1, dtype=int)
    if taus.size == 0:
        return pd.DataFrame(), pd.Series(dtype=float), lag_map

    # keep only rows with enough history for the max lag
    taus_model_mask = (taus - head_needed >= 0)
    if np.sum(taus_model_mask) > 0:
        taus_model = taus[taus_model_mask]
    else:
        taus_model = taus[-1:].copy() if taus.size > 0 else np.array([], dtype=int)

    if len(taus_model) == 0:
        return pd.DataFrame(), pd.Series(dtype=float), lag_map

    X_train_final = (
        X_eng.iloc[taus_model, :]
             .replace([np.inf, -np.inf], np.nan)
             .fillna(0.0)
    )
    y_train_final = y.shift(-1).iloc[taus_model].copy()  # y_{t+1} target

    return X_train_final, y_train_final, lag_map


# --- Job 1: Save full lagged matrix (lag 0 + best lag >0, max_lag=6) ---

print("\n" + "=" * 20)
print("--- STARTING JOB 1: Build and save full lagged feature matrix ---")
print("=" * 20)

t_ref_full = T - 2  # reference origin (as before)
lag_map_full = build_best_lag_map_global(
    X_ifo,
    y,
    max_lag=MAX_LAG,
    corr_spec=DEFAULT_CORR_SPEC,
)
print(
    f"Lag map built (lag 0 + best lag >0, max_lag={MAX_LAG}) based on t={t_ref_full} (global)."
)

X_eng_full = build_engineered_matrix(X_ifo, lag_map_full)
X_eng_full = X_eng_full.replace([np.inf, -np.inf], np.nan).fillna(0.0)
lagged_features_path = OUTPUT_DIR / "X_eng_full_lagged.parquet"
X_eng_full.to_parquet(lagged_features_path)
print(f"Full lagged feature matrix saved to: {lagged_features_path}  |  Shape: {X_eng_full.shape}")


# --- 5. Rolling importance + aggregation (robust) ---

def perform_rolling_importance(start_origin: int, end_origin: int, run_name: str) -> Tuple[pd.DataFrame, Dict[str, pd.DataFrame]]:
    all_importances: Dict[str, List[Dict[str, float]]] = {'EN': [], 'ET': [], 'LGBM': []}
    origins = range(start_origin, end_origin + 1)
    all_feature_names_seen: set[str] = set()

    for t in tqdm(origins, desc=f"Rolling Importance ({start_origin}-{end_origin})"):
        X_tr, y_tr, _ = run_feature_pipeline_for_importance(X_ifo, y, t, lag_map=lag_map_full)
        if X_tr.empty or y_tr.empty or len(y_tr) < 20:
            # too little data -> skip
            continue

        feature_names = X_tr.columns.tolist()
        all_feature_names_seen.update(feature_names)

        results_t: Dict[str, Dict[str, float]] = {}

        # EN
        try:
            model_en = ENModel(HP_EN).fit(X_tr, y_tr)
            coefs = np.abs(model_en._reg.coef_)
            results_t['EN'] = dict(zip(feature_names, coefs))
        except Exception as e:
            print(f"EN failed at t={t}: {e}")
            results_t['EN'] = {}

        # ET
        try:
            model_et = ETModel(HP_ET).fit(X_tr, y_tr)
            results_t['ET'] = model_et.get_feature_importances()
        except Exception as e:
            print(f"ET failed at t={t}: {e}")
            results_t['ET'] = {}

        # LGBM (gain importances; GPU/CPU depends on setup)
        try:
            n_train = len(y_tr)
            safe_min_child = max(1, min(HP_LGBM['min_child_samples'], (n_train - 1) // 2))

            lgbm = LGBMRegressor(
                n_estimators=HP_LGBM['n_estimators'],
                learning_rate=HP_LGBM['learning_rate'],
                num_leaves=HP_LGBM['num_leaves'],
                max_depth=-1,
                feature_fraction=HP_LGBM['feature_fraction'],
                bagging_fraction=1.0,
                bagging_freq=0,
                min_child_samples=safe_min_child,
                reg_lambda=0.0,
                random_state=42,
                n_jobs=1,
                verbose=-1,
                **LGBM_DEVICE_PARAMS,   # GPU/CPU selection
            )
            lgbm.fit(X_tr, y_tr)

            booster = lgbm.booster_
            if booster.num_trees() > 0:
                gain = booster.feature_importance(importance_type="gain")
                names = booster.feature_name()
                imp_map = {n: float(v) for n, v in zip(names, gain)}
                results_t['LGBM'] = {fname: imp_map.get(fname, 0.0) for fname in feature_names}
            else:
                results_t['LGBM'] = {}
        except Exception as e:
            print(f"LGBM failed at t={t}: {e}")
            results_t['LGBM'] = {}

        # Append results (may be empty; handled later)
        for m in ["EN", "ET", "LGBM"]:
            all_importances[m].append(results_t.get(m, {}))

    print(f"Rolling importance calculation ({start_origin}-{end_origin}) done.")

    if not all_feature_names_seen:
        print("Warning: no features collected during rolling importance.")
        return pd.DataFrame(), {}

    # Aggregate per model (robust to empty dicts)
    df_agg = pd.DataFrame(index=sorted(all_feature_names_seen))
    all_raw_dfs: Dict[str, pd.DataFrame] = {}

    def _build_and_save_raw(model_name: str):
        imp_list = all_importances[model_name]
        has_any = any(bool(d) for d in imp_list)
        if not has_any:
            df_empty = pd.DataFrame(columns=df_agg.index)
            raw_path = OUTPUT_DIR / f"raw_importance_{model_name}_{run_name}.csv"
            df_empty.to_csv(raw_path)
            all_raw_dfs[model_name] = df_empty
            df_agg[model_name] = 0.0
            return

        df_imp = pd.DataFrame.from_records(imp_list).fillna(0.0)
        df_imp = df_imp.reindex(columns=df_agg.index, fill_value=0.0)
        raw_path = OUTPUT_DIR / f"raw_importance_{model_name}_{run_name}.csv"
        df_imp.to_csv(raw_path)
        all_raw_dfs[model_name] = df_imp.copy()

        if df_imp.shape[1] == 0:
            df_agg[model_name] = 0.0
        else:
            df_agg[model_name] = df_imp.mean(axis=0)

    for model_name in ["EN", "ET", "LGBM"]:
        _build_and_save_raw(model_name)

    # Normalize each model column (sum=1 if >0)
    cols_present = [c for c in ["EN", "ET", "LGBM"] if c in df_agg.columns]
    for c in cols_present:
        s = df_agg[c]
        ssum = s.sum()
        df_agg[c] = (s / ssum) if ssum > 0 else 0.0

    # Mean importance across available models
    if cols_present:
        df_agg['Mean_Importance'] = df_agg[cols_present].mean(axis=1)
    else:
        df_agg['Mean_Importance'] = 0.0

    return df_agg, all_raw_dfs


# --- 6. Diverse Top-K selection ---

def select_diverse_top_k(
    df_agg_norm: pd.DataFrame,
    redundancy_threshold: float,
    cum_importance_threshold: float,
    t_ref: int
) -> List[str]:
    if df_agg_norm.empty:
        return []

    if 'Mean_Importance' not in df_agg_norm.columns:
        return []

    N = min(len(df_agg_norm), 200)
    top_n_features = df_agg_norm.sort_values('Mean_Importance', ascending=False).head(N).index.tolist()

    # Reference data for redundancy filter
    X_tr_ref, _, _ = run_feature_pipeline_for_importance(X_ifo, y, t_ref, lag_map=lag_map_full)
    valid_top_n = [f for f in top_n_features if f in X_tr_ref.columns]
    if not valid_top_n:
        return []

    X_top_n = X_tr_ref[valid_top_n].copy()

    # Local time indices for redundancy_reduce_greedy
    local_taus = np.arange(len(X_top_n))

    scores_top_n = df_agg_norm.loc[valid_top_n, 'Mean_Importance'].to_dict()

    # NOTE: The D-argument was removed from redundancy_reduce_greedy, so do not pass it here.
    diverse_features = redundancy_reduce_greedy(
        X_sel=X_top_n,
        corr_spec=DEFAULT_CORR_SPEC,
        taus=local_taus,
        redundancy_param=redundancy_threshold,
        scores=scores_top_n
    )

    if not diverse_features:
        return []

    diverse_scores = df_agg_norm.loc[diverse_features, 'Mean_Importance']
    total_importance = diverse_scores.sum()
    if total_importance <= 0:
        return diverse_features[:1]

    cumulative_pct = (diverse_scores.cumsum() / total_importance)
    k_dynamic = max(1, int(np.searchsorted(cumulative_pct.values, cum_importance_threshold, side='right')))
    return diverse_features[:k_dynamic]


# ======================================================================
# --- RUN: Ex-ante + ex-post (t=120 to T-2) ---
# ======================================================================
print("\n" + "=" * 20)
print("--- STARTING FULL IMPORTANCE RUN (t=120 to T-2) ---")
print("=" * 20)

START_ORIGIN_FULL = 120
END_ORIGIN_FULL   = T - 2

try:
    df_agg_norm_full, raw_dfs_full = perform_rolling_importance(START_ORIGIN_FULL, END_ORIGIN_FULL, "full_run")
except Exception as e:
    print("perform_rolling_importance crashed with:", repr(e))
    raise

# --- Output 1: Top-N per model (only if column exists) ---
if not df_agg_norm_full.empty:
    present_cols = [c for c in ['EN', 'ET', 'LGBM'] if c in df_agg_norm_full.columns]
    for model_name in present_cols:
        top_list = df_agg_norm_full.sort_values(model_name, ascending=False).head(TOP_N_PER_MODEL).index.tolist()
        output_path_model = OUTPUT_DIR / f"top_features_{model_name}_full.json"
        with open(output_path_model, "w") as f:
            json.dump(top_list, f, indent=2)
        print(f"Saved Top-{TOP_N_PER_MODEL} for {model_name} to: {output_path_model}")
else:
    print("No aggregated importances to produce Top-N lists.")

# --- Output 2: Diverse MEAN lists (only if Mean_Importance exists) ---
if not df_agg_norm_full.empty and 'Mean_Importance' in df_agg_norm_full.columns:
    thresholds_to_create = [0.50, 0.90, 0.99]
    print(f"\n--- Creating diverse 'MEAN' feature lists for thresholds: {thresholds_to_create} ---")
    for thresh in thresholds_to_create:
        final_top_features = select_diverse_top_k(
            df_agg_norm=df_agg_norm_full,
            redundancy_threshold=REDUNDANCY_THRESHOLD,
            cum_importance_threshold=thresh,
            t_ref=END_ORIGIN_FULL
        )
        thresh_str = str(thresh).replace(".", "_")
        output_path_mean = OUTPUT_DIR / f"top_mean_features_full_thresh_{thresh_str}.json"
        with open(output_path_mean, "w") as f:
            json.dump(final_top_features, f, indent=2)
        print(f"Saved mean diverse list (len={len(final_top_features)}) to: {output_path_mean}")
else:
    print("Mean_Importance not available; skipping diverse lists.")

# ======================================================================
# --- Job 3: Rolling mean importance (60m) over available models ---
# ======================================================================
print("\n" + "=" * 50)
print("--- STARTING JOB 3: Build and save Rolling Mean Importance (60m) ---")
print("=" * 50)

ROLLING_WINDOW_SIZE = 60
run_name = "full_run"

# Collect available raw files
raw_paths = {m: OUTPUT_DIR / f"raw_importance_{m}_{run_name}.csv" for m in ["EN", "ET", "LGBM"]}
available = [m for m, p in raw_paths.items() if p.exists()]

if not available:
    print("No raw importance files found. Skipping Job 3.")
else:
    dfs_norm = []
    union_cols = set()
    tmp_dfs = {}
    for m in available:
        df = pd.read_csv(raw_paths[m], index_col=0)
        tmp_dfs[m] = df
        union_cols.update(df.columns.tolist())

    union_cols = sorted(union_cols)

    # Row-wise normalization per model, reindex to union columns
    for m in available:
        df = tmp_dfs[m].reindex(columns=union_cols, fill_value=0.0)
        row_sums = df.sum(axis=1).replace(0.0, np.nan)
        df_norm = df.div(row_sums, axis=0).fillna(0.0)
        dfs_norm.append(df_norm)

    # Mean across available models (1..3)
    df_mean_imp_monthly = sum(dfs_norm) / len(dfs_norm)

    mean_raw_output_path = OUTPUT_DIR / f"raw_importance_MEAN_{run_name}.csv"
    df_mean_imp_monthly.to_csv(mean_raw_output_path)
    print(f"Saved raw time series importance for MEAN (combined) to: {mean_raw_output_path}")

    # Rolling 60m
    df_rolling_mean_imp = df_mean_imp_monthly.rolling(window=ROLLING_WINDOW_SIZE, min_periods=1).mean()

    # Try setting target dates (t+1) as index if lengths match
    target_dates = y.index[START_ORIGIN_FULL + 1 : END_ORIGIN_FULL + 2]
    if len(target_dates) == len(df_rolling_mean_imp):
        df_rolling_mean_imp.index = target_dates
        df_rolling_mean_imp = df_rolling_mean_imp.reindex(y.index)

        rolling_imp_path = OUTPUT_DIR / f"rolling_mean_importance_{ROLLING_WINDOW_SIZE}m.parquet"
        df_rolling_mean_imp.to_parquet(rolling_imp_path)
        print(f"Rolling Mean Importance saved to: {rolling_imp_path}  |  Shape: {df_rolling_mean_imp.shape}")
    else:
        print(
            f"Note: target_dates length ({len(target_dates)}) != rolling file length ({len(df_rolling_mean_imp)})."
        )
        print("Skipping parquet export; the raw MEAN CSV was saved.")

print("\nFeature Importance Analysis Complete.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



--- LightGBM Device Selection ---
LightGBM version: 4.6.0
GPU bevorzugt: True
=> GPU war gewünscht, aber nicht verfügbar/benutzbar.
   Grund: GPU-Probe fehlgeschlagen: LightGBMError('GPU Tree Learner was not enabled in this build.\nPlease recompile with CMake option -DUSE_GPU=1')
=> Fallback: Nutze CPU.


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1


PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Output Directory = /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/feature_importance/outputs_no_missing
INFO in load_ifo_features: Renaming columns to ensure validity.
Daten geladen. Shapes: (407, 2160) (407,)

--- STARTING JOB 1: Build and save full lagged feature matrix ---
Lag map (Lag 0 + bester Lag>0, max Lag=6) erstellt auf Basis von t=405 (global).
Full lagged feature matrix saved to: /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/feature_importance/outputs_no_missing/X_eng_full_lagged.parquet  |  Shape: (407, 4320)

--- STARTING FULL IMPORTANCE RUN (t=120 to T-2) ---


Rolling Importance (120-405):   0%|          | 0/286 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.364e-01, tolerance: 2.081e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
Rolling Importance (120-405):   0%|          | 0/286 [00:02<?, ?it/s]


KeyboardInterrupt: 